In [ ]:
!pip install earthaccess
!pip install xarray
!pip install cartopy
!pip install matplotlib
!pip install cftime
!pip install h5netcdf

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import earthaccess
import os
import xarray as xr
import h5py
import folium
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from matplotlib.colors import BoundaryNorm
%matplotlib inline
import glob

# **Aquisiçao dos dados**

In [ ]:
auth = earthaccess.login(persist=True)               #salvar os dados, para n ter q ficar logando toda vez que rodar o codigo

results = earthaccess.search_data(
    short_name="GPM_3IMERGDF",                         #GPM_3IMERGDF - Trabalha com dados por dia, GPM_3IMERGM - trabalha com dados a cada 30m
    version = '07',
    temporal = ('2024-01-01', '2024-01-30'),          #pegar dados pro mes de janeiro de 2024
    bounding_box = (-74, -34, -34, 5)                 #coordenadas que pega o brasil

)

print(len(results))

# Fazer o download para a pasta 'data'
arquivos_baixados = earthaccess.download(results, local_path='data')

# Verificar arquivos baixados
print("Arquivos baixados:")
print(arquivos_baixados)

#Caso for utilizar o de 30m, utilizar o codigo abaixo
# Filtrar arquivos com extensão .HDF5 ou .nc na pasta 'data'
#arquivos = [f for f in arquivos_baixados if f.endswith('.HDF5') or f.endswith('.nc')]

#Caso for utilizar o de Dia, utilizar o codigo abaixo
# Filtrar arquivos com extensão .HDF5 ou .nc4 na pasta 'data'
#rquivos = [f for f in arquivos_baixados if f.endswith('.HDF5') or f.endswith('.nc4')]
arquivos = sorted(glob.glob('data/3B-DAY.MS.MRG.3IMERG.202401*.nc4'))

# Tentar abrir o primeiro arquivo
if arquivos:
    ds = xr.open_dataset(arquivos[0])
    print(ds)
else:
    print("Nenhum arquivo .HDF5 ou .nc encontrado.")

In [ ]:
# Caso para 30m
'''
# Explorar os grupos e variáveis internas (caso seja HDF5 com grupos)
with h5py.File(arquivos[0], 'r') as f:
    def listar_grupos(nome, obj):
        if isinstance(obj, h5py.Dataset):
            print("Dataset:", nome)
        elif isinstance(obj, h5py.Group):
            print("Grupo:", nome)

    f.visititems(listar_grupos)
'''
#Caso para o Dia
# Abre o arquivo HDF5 e lista os grupos e datasets
with h5py.File(arquivos[0], 'r') as f:
    def listar_grupos(nome, obj):
        if isinstance(obj, h5py.Group):
            print(f"📁 Grupo: {nome}")
        elif isinstance(obj, h5py.Dataset):
            print(f"📄 Dataset: {nome} - shape: {obj.shape} - dtype: {obj.dtype}")

    print("📚 Estrutura do arquivo HDF5:")
    f.visititems(listar_grupos)

In [ ]:
# Abrir o arquivo com xarray
#ds = xr.open_dataset(arquivos[0], decode_times=True)
ds = xr.open_mfdataset(arquivos, combine='by_coords')

# Recorta apenas a área do Brasil.   Utilizei para reduzir a ram é conseguir rodar o codigo no google colab
ds_brasil = ds.sel(
    lon=slice(-74, -34),   # longitude: Oeste (negativo)
    lat=slice(-34, 5)      # latitude: Sul até Norte (de baixo pra cima!)
)

# Selecionar a variável de interesse
precip = ds_brasil['precipitation']  # ou, se preferir outra como MWprecipitation

# Converter para DataFrame
df = precip.to_dataframe().reset_index()
df

# Pre processamento e analise descritiva

In [ ]:
# Remoção dos valores ausentes (NaN) da coluna de precipitação
df = df.dropna(subset=['precipitation'])
df

In [ ]:
df.describe()

In [ ]:
df['time'].value_counts()

# **Analise dos dados**

In [ ]:
# Coordenadas da estação
lat_estacao = -9.3664
lon_estacao = -36.4194
buffer = 0.1  # tamanho do quadrado (graus)

# Filtrando o DataFrame
df_filtrado = df[(df['lat'] >= lat_estacao - buffer) & (df['lat'] <= lat_estacao + buffer) &
    (df['lon'] >= lon_estacao - buffer) & (df['lon'] <= lon_estacao + buffer)
]
df_filtrado

In [ ]:
df_filtrado['precipitation'].describe()

In [ ]:
df_agrupado=df_filtrado.groupby(['lon','lat'])['precipitation'].agg(['mean','sum','min','max'])
df_agrupado

# **Estação de dados da ATV 2**

In [ ]:
#Lendo o banco de dados

#liberando o acesso aos arquivos do drive
from google.colab import drive
drive.mount('/content/drive')

#lendo o banco de dados (Dados pluviometricos)
banco_de_dados = pd.read_excel('/content/drive/MyDrive/df_atv2.xlsx')
banco_de_dados

In [ ]:
#Filtrando os dados para o periodo de janeiro de 2024
banco_de_dados_filtrado = banco_de_dados[(banco_de_dados['Data']>='2024-01-01') & (banco_de_dados['Data']<='2024-01-30')]
banco_de_dados_filtrado


In [ ]:
banco_de_dados_agrupado=banco_de_dados_filtrado.groupby(['Longitude','Latitude'])['Valor'].agg(['mean','sum','min','max'])
banco_de_dados_agrupado

# **Comparação dos dados estatisticos**

In [ ]:
comparando = pd.concat([banco_de_dados_agrupado, df_agrupado])
comparando

# **Geração do mapa a partir dos dados**

In [ ]:
df

In [ ]:
# Gráfico para os dados gerais do Brasil

# Primeiro, converta os dados em grade (malha 2D)
lat = np.sort(df['lat'].unique())
lon = np.sort(df['lon'].unique())
precip_grid = df.pivot_table(index='lat', columns='lon', values='precipitation').values

# Configuração do gráfico
fig = plt.figure(figsize=(21, 7))
ax = plt.axes(projection=ccrs.PlateCarree())

# Costas e grade
ax.coastlines(resolution="110m", linewidth=1)
gl = ax.gridlines(draw_labels=True, linewidth=1, color='black', linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = True
gl.xlocator = mticker.FixedLocator([-180, -90, 0, 90, 180])
gl.ylocator = mticker.FixedLocator([-60, -50, -25, 0, 25, 50, 60])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 16, 'color': 'black'}
gl.ylabel_style = {'size': 16, 'color': 'black'}

# Níveis de contorno e normalização da coloração
clevs = np.arange(0, 50 + 0.1, 0.2)
norm = BoundaryNorm(clevs, ncolors=plt.cm.rainbow.N, clip=True)

# Mapa de cores
mesh = ax.pcolormesh(lon, lat, precip_grid, cmap=plt.cm.rainbow, norm=norm, shading='auto')

# Ponto da estação
ax.plot(lon_estacao, lat_estacao, marker='o', color='white', markersize=5, label='Estação')

# Legenda
ax.legend(loc='upper right', fontsize=14)

# Barra de cores
cb = plt.colorbar(mesh, ax=ax, orientation="vertical", pad=0.05, aspect=16, shrink=0.8)
cb.set_label('Precipitação (mm/d)', size=18)
cb.ax.tick_params(labelsize=14)

# Título
plt.title('GPM IMERG Janeiro 2024', size=24)

plt.show()


In [ ]:
#Grafico para os dados perto da estaçao

# Primeiro, converta os dados em grade (malha 2D)
lat = np.sort(df['lat'].unique())
lon = np.sort(df['lon'].unique())
precip_grid = df.pivot_table(index='lat', columns='lon', values='precipitation').values

# Configuração do gráfico
fig = plt.figure(figsize=(21, 7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-30, -50, -15, 0], crs=ccrs.PlateCarree())
#ax.set_extent([-36.5194, -36.3194, -9.4664, -9.2664], crs=ccrs.PlateCarree())
# Costas e grade
ax.coastlines(resolution="110m", linewidth=1)
gl = ax.gridlines(draw_labels=True, linewidth=1, color='black', linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = True
gl.xlocator = mticker.FixedLocator([-180, -90, 0, 90, 180])
gl.ylocator = mticker.FixedLocator([-60, -50, -25, 0, 25, 50, 60])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 16, 'color': 'black'}
gl.ylabel_style = {'size': 16, 'color': 'black'}


# Níveis de contorno e normalização da coloração
clevs = np.arange(0, 50 + 0.1, 0.2)
norm = BoundaryNorm(clevs, ncolors=plt.cm.rainbow.N, clip=True)

# Mapa de cores
mesh = ax.pcolormesh(lon, lat, precip_grid, cmap=plt.cm.rainbow, norm=norm, shading='auto')

# Ponto da estação
ax.plot(lon_estacao, lat_estacao, marker='o', color='white', markersize=5, label='Estação')

# Legenda
ax.legend(loc='upper left', fontsize=14)

# Barra de cores
cb = plt.colorbar(mesh, ax=ax, orientation="vertical", pad=0.05, aspect=16, shrink=0.8)
cb.set_label('Precipitação (mm/d)', size=18)
cb.ax.tick_params(labelsize=14)

# Título
plt.title('GPM IMERG Janeiro 2024', size=24)

plt.show()
